In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title <font size="5">← ឵឵<i>Install Ffmpeg</font> { vertical-output: true }
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
os.system('pip install ffpb')
clear_output()
print('Installation finished.')

In [ ]:
mappedFolder = "gdrive/My Drive/GitHub/Lecture Video Online Ready/"

In [ ]:
cd $mappedFolder

#Define function

In [ ]:
from datetime import datetime
FMT = '%H:%M:%S'

def makeReady(disclaimerPath, disclaimerDuration, logoPath, videoPath, trimBefore, trimAfter):
  vWidth      = !ffprobe $videoPath -v error -show_entries stream=width
  vHeight     = !ffprobe $videoPath -v error -show_entries stream=height
  vFps        = !ffprobe $videoLecture -v error -show_entries stream=r_frame_rate
  videoWidth  = vWidth[1].replace("width=","")
  videoHeight = vHeight[1].replace("height=","")
  videoRes    = videoWidth + "x" + videoHeight
  vidFps      = vFps[1].replace("r_frame_rate=","")
  videFps     = vidFps.split("/")
  videoFps    = int(videFps[0]) / int(videFps[1])
  discFrames  = round(videoFps, 3) * int(disclaimerDuration)

  start_time    = trimBefore
  start_seconds = datetime.strptime(str(trimBefore), "%H:%M:%S") - datetime(1900, 1, 1)
  starts = str(int(start_seconds.total_seconds()))

  end_time    = trimAfter
  end_seconds = datetime.strptime(str(trimAfter), "%H:%M:%S") - datetime(1900, 1, 1)
  ends = str(int(end_seconds.total_seconds()))

  tDelta = datetime.strptime(end_time, FMT) - datetime.strptime(start_time, FMT)
  videoFadeOut = str(int(tDelta.total_seconds()) - 1) # minus fade duration
  duration = datetime.strptime(str(tDelta), "%H:%M:%S")
  a_timedelta = duration - datetime(1900, 1, 1)

  os.environ['videoPath']     = videoPath
  os.environ['videoOutput']   = "out." + videoPath
  os.environ['videoWidth']    = str(videoWidth)
  os.environ['videoHeight']   = str(videoHeight)
  os.environ['videoFps']      = str(round(videoFps, 3))
  os.environ['videoRes']      = videoRes
  os.environ['logoPath']      = logoPath

  os.environ['disclaimerPath']= disclaimerPath
  os.environ['discFrames']    = str(round(discFrames))
  os.environ['discFadeOut']   = str(disclaimerDuration - 1)

  os.environ['starts']        = str(starts)
  os.environ['ends']          = str(ends)
  os.environ['tDelta']        = str(int(a_timedelta.total_seconds()))
  os.environ['videoFadeOut']  = str(videoFadeOut)

  os.system('ffmpeg -i "$videoPath" -i $logoPath -i $disclaimerPath -f lavfi -t 1 -i anullsrc -filter_complex "[2:v]fps=fps=$videoFps,loop=loop=$discFrames:size=$videoFps:start=0[v2];[v2]fade=t=in:st=0:d=1,fade=t=out:st=$discFadeOut:d=1[vv2];[0:v]trim=start=$starts:end=$ends,setpts=PTS-STARTPTS,scale=$videoWidth:$videoHeight[v0];[0:a]atrim=start=$starts:end=$ends,asetpts=PTS-STARTPTS[a0];[v0][1:v]overlay=0:0[vv0];[vv0]fade=t=in:st=0:d=1,fade=t=out:st=$videoFadeOut:d=1[vvv0];[a0]afade=t=in:st=0:d=1,afade=t=out:st=$videoFadeOut:d=1[aa0];[vv2][3:a][vvv0][aa0]concat=n=2:v=1:a=1[vv][aa]" -map [vv] -map [aa] -c:a aac -vcodec libx264 -s $videoRes $videoOutput')


#Option 1: Run as function

In [ ]:
makeReady("disclaimer.jpg", 5, "logo.png", "lecture.mp4", "00:00:15", "00:00:25")

#Option 2: Set trimming points and start render

In [ ]:
from datetime import datetime
FMT = '%H:%M:%S'
#@title <font size="5">← ឵឵<i>Trim,add logo, add fade-in/out, and disclaimer to lecture</font> { vertical-output: true }
lecture_file_path = "lecture.mp4" #@param {type:"string"}
output_file_name = "final_" + lecture_file_path

vFps        = !ffprobe $inputFile -v error -show_entries stream=r_frame_rate
vidFps      = vFps[1].replace("r_frame_rate=","")
videFps     = vidFps.split("/")
videoFps    = int(videFps[0]) / int(videFps[1])

disclaimer_duration = "5" #@param {type:"string"}
discTotalFrames = int(disclaimer_duration) * videoFps

start_time = "00:00:14" #@param {type:"string"}
start_seconds = datetime.strptime(str(start_time), "%H:%M:%S") - datetime(1900, 1, 1)
starts = str(int(start_seconds.total_seconds()))

end_time = "00:38:04" #@param {type:"string"}
end_seconds = datetime.strptime(str(end_time), "%H:%M:%S") - datetime(1900, 1, 1)
ends = str(int(end_seconds.total_seconds()))

tDelta = datetime.strptime(end_time, FMT) - datetime.strptime(start_time, FMT)
tDelta2 = str(int(tDelta.total_seconds()) - 2) # minus fade duration
duration = datetime.strptime(str(tDelta), "%H:%M:%S")
a_timedelta = duration - datetime(1900, 1, 1)

os.environ['inputFile'] = lecture_file_path
os.environ['outputFile'] = output_file_name
os.environ['discTotalFrames'] = discTotalFrames

os.environ['starts'] = starts
os.environ['ends'] = ends
os.environ['tDelta'] = str(int(a_timedelta.total_seconds()))
os.environ['tDelta2'] = tDelta2

loadingAn(name="lds")
textAn("Trimming, adding logo, adding fade-in/out, adding disclaimer...", ty='twg')
os.system('ffmpeg -i "$inputFile" -i logo.png -i disclaimer.jpg -f lavfi -t 1 -i anullsrc -filter_complex "[2:v]fps=fps=$videoFps,loop=loop=$discTotalFrames:size=$videoFps:start=0[v2];[v2]fade=t=in:st=0:d=1,fade=t=out:st=14:d=1[vv2];[0:v]trim=start=$starts:end=$ends,setpts=PTS-STARTPTS,scale=1920:1080[v0];[0:a]atrim=start=$starts:end=$ends,asetpts=PTS-STARTPTS[a0];[v0][1:v]overlay=0:0[vv0];[vv0]fade=t=in:st=0:d=1,fade=t=outst=$tDelta2:d=1[vvv0];[a0]afade=t=in:st=0:d=1,afade=t=out:st=$tDelta2:d=1[aa0];[vv2][3:a][vvv0][aa0]concat=n=2:v=1:a=1[vv][aa]" -map [vv] -map [aa] -c:a aac -vcodec libx264 -s 1920x1080 $outputFile')
clear_output()
print('Trimmed lecture, added logo, fade-in/out and disclaimer.')
